# 1. Introduction

Percolation is a simple model in statistical physics that has been well studied. It has a wide range of uses including modelling resistor networks, ecological disturbances (like forest fires), epidemics, and was first used to model fluid flow through porous materials.
<br><br> Percolation theory describes the behaviour of _clusters_ in a large lattice of dimension $d$. The two most studied percolation models are site percolation and bond percolation. In site percolation, the lattice vertices are the relevant quantitities. Each lattice vertex/site is either "occupied", with probability $p$, or "unoccupied", with probability 1-$p$. A cluster is a group of occupied lattice sites that are connected by a chain of nearest neigbour links (i.e for every site in a cluster, one of its nearest neigbours is also occupied) [1]. 
<br><br> Bond percolation is a similar model where the lattice bonds are the relevant quantities. Each bond is either "occupied", with probability $p$, or "unoccupied", with probability 1-$p$. In both models, the probability of a site/bond being occupied is independent of the status of its neighbours. The system is said to _percolate_ when the largest cluster spans the lattice (i.e. it extends from one lattice boundary to the opposite boundary).
<br><br>The critical proabibility value, $p_{c}$, is the probability at which the system can first percolate. For bond percolation in two dimensions, $p_{c} = \frac{1}{2}$. No analytic expression exists for site percolation in two dimensions, or for either model in 3 dimensions or above [2]. 
<br><br>Percolation is a random process. Different percolation lattices will contain clusters of different shapes and sizes - we wish to discuss their average properties. Hence, We often want to find the average value of some observable $Q(p)$ (e.g. average cluster size) over a range of values of $p$. In this report we will explore the most popular algorithm for doing this, the Newman-Ziff algorithm.


# 2. Percolation Algorithms

### 2.1 Conventional Algoritm

Let us first consider the conventional direct percolation algorithm. Consider some observable $Q(p)$ that we wish to calculate over a range of values of $p$. We must perform simulations at many closely spaced values of $p$, and interpolate between measurements to obtain a continous curve of $Q(p)$. This, however, introduces error into the results and it is better instead to measure $Q$ for a fixed number $n$ of occupied sites in the range of interest. The probability of there being exactly $n$ occupied sites out of a possible $N$ sites is given by the Binomial distribution:
<br>$$B(N,n,p) = \binom{N}{n}p^{n}(1-p)^{N-n}$$ 
If we measure the observable for all values of $n$ (giving a set of measurements {$Q_{n}$}) we can find $Q(p)$ for all $p$:
<br>$$Q(p)=\sum\limits_{n=0}^{N}B(N,n,p)Q_{n}$$
The same expression applies for bond percolation but replacing $N$ with $M$, where $M$ is the total number of bonds.
<br><br>For site percolation, starting with an empty lattice of $N$ sites and $M$ bonds , it takes time $O(N)$ to fill the lattice and time $O(M)$ to find all the clusters for each value of $n$. Hence, the calculation takes time $O(M+N)$ for each value of $n$ and so $O(N^{2}+MN)$ overall. For a regular lattice $M=\frac{1}{2}zN$, where $z$ is the co-ordination number, and so $O(N^{2}+MN)$ is equivalent to $O(N^{2})$ . The algorithm complexity for bond percolation is the same as for site percolation on a regular lattice. [2]

### 2.2 Newmann-Ziff Algorithm
The Newman-Ziff Algorithm takes time $O(N)$ which is a large improvement on the $O(N^{2})$ complexity of the conventional algorithm. 
<br><br>In the conventional percolation algorithm, a new state of the lattice with $n$ occupied sites/bonds is created for each value of $n$. However, since to measure $Q(p)$ we generate states for all values of $n$ from zero to $N$ then we can save time using the fact that a correct state with $n+1$ occupied sites/bonds can be derived from a correct state with $n$ occupied sites/bonds by adding one extra randomly chosen site/bond. The whole set of percolation states is derived from an empty lattice by adding sites/bonds one by one. This is the idea at the core of the algorithm. [3]
<br><br>Consider the case of bond percolation. We initially have an empty lattice with all $M$ bonds unoccupied so that each site is a cluster of size 1. To begin, we must randomly choose the order which the bonds are occupied. A simple way to do this is as follows:  
1. Create a list of the bonds with positions labelled from 1 to $M$
2. For $i=1$, choose a number $j$ randomly in the range $i\le j\le M$ 
3. Exchange the bonds at positions $i$ and $j$
4. Repeat this for all $i$ in the range $1\le i\le M$

This generates all possible permutations of bond orders with equal probability and has complexity $O(M)$ [2].
<br><br>We can then occupy the bonds in the order that has been chosen. To keep track of the cluster configuration of the lattice we use a _union/find_ algorithm. When a new bond is occupied, these algorithms find the clusters the sites at either end of the bond are part of. If the sites are part of different clusters, the two clusters are combined into a single cluster.
In their paper of 2001 Newman and Ziff discuss different union/find algorithms. The most efficient one is a weighted union/find algorithm with path compression.
<br><br>This makes use of _tree data structures_. These are hierarchical structures that are used to represent data in a way that is easy to navigate. They consist of nodes connected by edges in a hierarchical relationship with the _root_ being the topmost node of the tree. In our case, each cluster is stored as a separate tree with each site in the cluster corresponding to a node. Each cluster has one root site (which is the root of the tree) and all other sites have a _pointer_ that points towards the root or towards another site in the cluster so that by following the pointers we can go from any site in the cluster to the root of that cluster. 
<br><br>When a new bond is added, the pointers belonging to the sites at each end of the bond are followed. If the pointers lead to the same root site, then the sites are part of the same cluster. After the path is traversed, each pointer along the path is changed to point directly to the root of the cluster. This is called _path compression_ and it speeds up the traversal of the cluster. If instead the pointers lead to different roots (so the sites are part of different clusters), the two clusters are combined by adding a pointer from the root of one cluster to the root of the other. The pointer is always orientated to point towards the larger of the two clusters. To do this, the size of a cluster (i.e number of sites) is stored at the root site of each cluster. When two clusters are combined, the size of the larger cluster is updated by adding to it the size of the smaller cluster. This process is repeated until all bonds in the lattice are occupied [2].
<br><br>It can be shown that time taken for the union/find algorithm is $O(1)$ in lattice size. Therefore, it takes time $O(1)$ to occupy each bond and so time $O(M)$ to add all the bonds (whilst keeping track of all the clusters). Since it also takes time $O(M)$ to generate the order in which bonds are added, for bond percolation, the Newman Ziff algorithm has overall complexity $O(M)$. The algorithm is very similar for site percolation (the sites are occupied in a random order instead of the bonds) and, for a regular lattice, has complexity $O(N)$ [2].




# 3. Implementation of Newman-Ziff algorithm
We will now implement the Newman-Ziff algorithm. The code is based of that given in appendix A of the paper by Newman and Ziff (written in C) but has been modified for python [2]. The programme is for site percolation on a square lattice with $N=L\times{L}$ sites and periodic boundary conditions. 

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import random

L = 64
N = L**2
Empty = -N-1

Sites are indexed with an integer label, taking values from 0 to $N-1$.
 <br><br>We first construct an array "ptr" which contains the pointer for each lattice site and stores the size of each cluster at its root site. In the array, each non-root occupied site is labelled with the index of its parent site in the tree (i.e the site its pointer points towards). Root sites are labelled with a value that is equal to minus the size of the cluster (the negative means root sites can be easily distinguished from non-root sites). All unoccupied sites takes the value of "Empty".  

In [ ]:
ptr = np.zeros(N, dtype=int)    #array of pointers

We define a function "nearestneighbour" which generates the nearest neigbours of each site and stores them in an array "nn".

In [ ]:
nn = np.zeros((N,4), dtype=int) #array of nearest neighbours

def nearestneighbour():

    """constructs a numpy.ndarray of the nearest neighbours of each site in a square lattice
    with N=L*L sites and periodic boundary conditions"""

    for i in range(N):
        nn[i,0] = (i+1)%N
        nn[i,1] = (i-1)%N
        nn[i,2] = (i+L)%N
        nn[i,3] = (i-L)%N

        if i%L==0:
            nn[i,1]=i+L-1
       
        if (i+1)%L==0:
            nn[i,0]=i-L+1



We also construct a function "permutate" which creates a random permutation of the site labels and stores them in an array "order". This is the order in which the sites will be occupied.

In [ ]:
order = np.zeros(N, dtype=int)  #occupation order array

def permutate():

    """constructs a random permutation of the site labels (integers from 0 to N-1) using the algorithm 
    given earlier and stores it in a numpy.ndarray """
    
    j=0
    temp=0
    
    for i in range(N):
        order[i]=i

    for i in range(N):
        
        #generates a random number j between i and N
        j = random.randint(i,N-1)
        
        #swaps the positions of sites i and j
        temp=order[i]
        order[i] = order[j]
        order[j] = temp

The "findroot" function returns the (label of) root site of the cluster each site is part of and also carries out path compression.

In [ ]:
def findroot(i):

    """Returns the label (an integer from 0 to N-1) of the root site of the cluster 
    that the input site (label i) is part of. Also carries out path compression by 
    setting the value of ptr[i] to be the label of the root site. """
    
    j=k=i
    
    while ptr[j]>=0:
        ptr[k]=ptr[j]
        k=j
        j=ptr[j]
                
    return j
     

#### __References__

[1] D. Stauffer and A. Aharony, _Introduction to Percolation Theory_, 2nd ed. Taylor and Francis, London, 1992
<br>[2] Newman, MEJ, and Robert M Ziff. 2000. “Efficient Monte Carlo Algorithm and High-Precision Results for Percolation.” _Physical Review Letters_ 85 (19): 4104.
<br>[3] Sorge, A. 2015. _The Newman–Ziff Algorithm pypercolate documentation_. Available at: https://pypercolate.readthedocs.io/en/stable/newman-ziff.html 